In [82]:
#Code Reference : https://qiskit.org/documentation/_modules/qiskit/finance/applications/ising/portfolio.html#get_operator
from qiskit import Aer
from qiskit.circuit.library import TwoLocal
from qiskit.aqua import QuantumInstance
from qiskit.finance.applications.ising import portfolio
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.finance.data_providers import RandomDataProvider
from qiskit.aqua.algorithms import VQE, QAOA, NumPyMinimumEigensolver
from qiskit.aqua.components.optimizers import COBYLA
import numpy as np
import matplotlib.pyplot as plt
import datetime

import gurobipy as gp
from gurobipy import GRB
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import datetime
import datetime
import pandas as pd

/opt/anaconda3/lib/python3.8/site-packages/qiskit/finance/__init__.py:50: DeprecationWarning: The package qiskit.finance is deprecated. It was moved/refactored to qiskit_finance (pip install qiskit-finance). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('finance', 'qiskit_finance', 'qiskit-finance')
/opt/anaconda3/lib/python3.8/site-packages/qiskit/optimization/__init__.py:92: DeprecationWarning: The package qiskit.optimization is deprecated. It was moved/refactored to qiskit_optimization (pip install qiskit-optimization). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('optimization', 'qiskit_optimization', 'qiskit-optimization')


In [83]:
df0 = pd.read_csv("Amazon_pred.csv")
df1 = pd.read_csv("AMD_pred.csv")
df2 = pd.read_csv("Apple_pred.csv")
df3= pd.read_csv("Cisco_pred.csv")
df4= pd.read_csv("FB_pred.csv")
df5= pd.read_csv("Microsoft_pred.csv")
df6 = pd.read_csv("Qualcomm_pred.csv")
df7 = pd.read_csv("Starbuck_pred.csv")
df8 = pd.read_csv("Tesla_pred.csv")
df9 = pd.read_csv("ZNGA_pred.csv")
df0=df0.drop(df0.columns[0], axis=1)
df1=df1.drop(df1.columns[0], axis=1)
df2=df2.drop(df2.columns[0], axis=1)
df3=df3.drop(df3.columns[0], axis=1)
df4=df4.drop(df4.columns[0], axis=1)
df5=df5.drop(df5.columns[0], axis=1)
df6=df6.drop(df6.columns[0], axis=1)
df7=df7.drop(df7.columns[0], axis=1)
df8=df8.drop(df8.columns[0], axis=1)
df9=df9.drop(df9.columns[0], axis=1)
df1

,Apple_new
0,101.300370
1,115.045775
2,113.272175
3,121.401178
4,124.061579
...,...
184,107.655772
185,109.577173
186,112.976575
187,133.816383


In [84]:
df=pd.concat([df0, df1,df2,df3,df4,df5,df6,df7,df8,df9], axis=1, ignore_index=True)
df.columns = ['Amazon',  'AMD',  'Apple',  'Cisco',  'FB',  'Microsoft', 
       'Qualcomm', 'Starbuck', 'Tesla', 'ZNGA']
df.tail()

,Amazon,AMD,Apple,Cisco,FB,Microsoft,Qualcomm,Starbuck,Tesla,ZNGA
184,3228.534698,107.655772,3021.183301,43.063046,285.233006,221.152058,152.642933,94.493827,680.484727,9.591525
185,3048.913791,109.577173,3258.967080,43.848041,266.481544,240.070737,109.251466,105.841186,367.242886,10.631407
186,3083.738253,112.976575,3071.622285,43.641463,250.739576,248.837930,128.995731,110.050690,521.953795,9.057141
187,3148.805010,133.816383,3297.877152,45.955133,245.878086,216.537746,134.046590,72.897241,306.122527,10.089802
188,3245.030496,126.574180,3194.116958,45.087506,256.295564,203.156242,100.527256,88.454104,430.273256,9.721510


In [85]:
stocks = df.columns

In [88]:
#Calculating the mean vector
mu = df.mean()

In [89]:
mu

Amazon       3164.651061
AMD           120.440869
Apple        3144.509094
Cisco          45.214076
FB            271.443473
Microsoft     223.666736
Qualcomm      130.153372
Starbuck       95.843735
Tesla         537.809920
ZNGA            9.808931
dtype: float64

In [90]:
mu=np.array([3164.651061,  120.440869, 3144.509094,45.214076, 271.443473,223.666736,130.153372,95.843735, 537.809920, 9.808931])

In [91]:
mu

array([3164.651061,  120.440869, 3144.509094,   45.214076,  271.443473,
        223.666736,  130.153372,   95.843735,  537.80992 ,    9.808931])

In [97]:
#saving the covariance of the stock data
stock_cov=df.cov()

In [98]:
#saving the matrix as an array
sigma = np.array(stock_cov)
sigma

array([[ 4.27765434e+03, -8.46610601e+01,  5.49111690e+02,
        -1.37078860e+01,  4.63930122e+01,  6.59643796e-01,
        -2.45991249e+01, -8.25580959e+00,  5.96070120e+02,
         2.55181823e+00],
       [-8.46610601e+01,  1.18164560e+02,  1.00344349e+02,
         1.68534320e+00, -2.06606470e+01, -2.12937613e+01,
        -8.54127244e+00, -2.45767936e+00, -2.06351602e+02,
         3.32851457e-01],
       [ 5.49111690e+02,  1.00344349e+02,  1.13802300e+04,
         1.17358132e+01, -9.05065955e+01, -4.35778248e+00,
        -1.28801388e+02,  1.49427336e+02, -2.60669871e+02,
         2.57085333e+00],
       [-1.37078860e+01,  1.68534320e+00,  1.17358132e+01,
         9.25538083e+00,  1.29314090e-01, -3.39297375e+00,
        -1.25744706e+00, -3.90842656e+00,  3.29342145e+01,
        -3.81598537e-02],
       [ 4.63930122e+01, -2.06606470e+01, -9.05065955e+01,
         1.29314090e-01,  3.00805927e+02, -2.32025543e+01,
         3.19580665e+01, -2.25130284e+01,  1.97618575e+02,
        -6.

In [99]:
# qiakit library

num_assets=10
q = 0.5                   # set risk factor
budget = num_assets // 2  # set budget
penalty = num_assets 
qubitOp, offset = portfolio.get_operator(mu, sigma, q, budget, penalty)

/opt/anaconda3/lib/python3.8/site-packages/qiskit/finance/applications/ising/portfolio.py:84: DeprecationWarning: The package qiskit.aqua.operators is deprecated. It was moved/refactored to qiskit.opflow (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  return WeightedPauliOperator(paulis=pauli_list), offset


In [100]:
def index_to_selection(i, num_assets):
    s = "{0:b}".format(i).rjust(num_assets)
    x = np.array([1 if s[i]=='1' else 0 for i in reversed(range(num_assets))])
    return x

def print_result(result):
    selection = sample_most_likely(result.eigenstate)
    value = portfolio.portfolio_value(selection, mu, sigma, q, budget, penalty)
    print('Optimal: selection {}, value {:.4f}'.format(selection, value))

    eigenvector = result.eigenstate if isinstance(result.eigenstate, np.ndarray) else result.eigenstate.to_matrix()
    probabilities = np.abs(eigenvector)**2
    i_sorted = reversed(np.argsort(probabilities))
    print('\n----------------- Full result ---------------------')
    print('selection\tvalue\t\tprobability')
    print('---------------------------------------------------')
    for i in i_sorted:
        x = index_to_selection(i, num_assets)
        value = portfolio.portfolio_value(x, mu, sigma, q, budget, penalty)
        probability = probabilities[i]
        print('%10s\t%.4f\t\t%.4f' %(x, value, probability))

In [101]:
#classical simulation
exact_eigensolver = NumPyMinimumEigensolver(qubitOp)
result = exact_eigensolver.run()

print_result(result)

/opt/anaconda3/lib/python3.8/site-packages/qiskit/aqua/algorithms/minimum_eigen_solvers/minimum_eigen_solver.py:36: DeprecationWarning: The package qiskit.aqua.algorithms.minimum_eigen_solvers is deprecated. It was moved/refactored to qiskit.algorithms.minimum_eigen_solvers (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('aqua.algorithms.minimum_eigen_solvers',
/opt/anaconda3/lib/python3.8/site-packages/qiskit/aqua/algorithms/eigen_solvers/eigen_solver.py:36: DeprecationWarning: The package qiskit.aqua.algorithms.eigen_solvers is deprecated. It was moved/refactored to qiskit.algorithms.eigen_solvers (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('aqua.algorithms.eigen_solvers',
/opt/anaconda3/lib/python3.8/site-packages/qiskit/aqua/operators/state_fns/vector_state_fn.py:215: DeprecationWarnin

Optimal: selection [1 1 0 1 1 1 0 1 0 0], value -1489.4746

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
[1 1 0 1 1 1 0 1 0 0]	-1489.4746		1.0000
[1 1 1 1 1 1 1 1 1 1]	10984.4952		0.0000
[0 0 0 0 1 0 1 0 1 0]	9136.0996		0.0000
[1 0 1 1 1 0 1 0 1 0]	11302.1271		0.0000
[0 0 1 1 1 0 1 0 1 0]	11164.6832		0.0000
[1 1 0 1 1 0 1 0 1 0]	8295.7637		0.0000
[0 1 0 1 1 0 1 0 1 0]	8792.0925		0.0000
[1 0 0 1 1 0 1 0 1 0]	8665.6515		0.0000
[0 0 0 1 1 0 1 0 1 0]	9097.3193		0.0000
[1 1 1 0 1 0 1 0 1 0]	11031.6506		0.0000
[0 1 1 0 1 0 1 0 1 0]	10965.1599		0.0000
[1 0 1 0 1 0 1 0 1 0]	11302.8795		0.0000
[0 0 1 0 1 0 1 0 1 0]	11171.7277		0.0000
[1 1 0 0 1 0 1 0 1 0]	8306.5666		0.0000
[0 1 0 0 1 0 1 0 1 0]	8809.1875		0.0000
[1 0 0 0 1 0 1 0 1 0]	8698.1397		0.0000
[1 1 1 1 0 0 1 0 1 0]	10978.6924		0.0000
[0 0 0 0 0 0 0 0 1 0]	9296.3452		0.0000
[0 1 1 1 0 0 1 0 1 0]	10972.3025		0.0000
[1 0 1 1 0 0 1 0 1 0]	11227.5752		0.0

In [102]:
# quantum vqe simulation

backend = Aer.get_backend('statevector_simulator')
seed = 50

cobyla = COBYLA()
cobyla.set_options(maxiter=500)
ry = TwoLocal(qubitOp.num_qubits, 'ry', 'cz', reps=3, entanglement='full')
vqe = VQE(qubitOp, ry, cobyla)
vqe.random_seed = seed

quantum_instance = QuantumInstance(backend=backend, seed_simulator=seed, seed_transpiler=seed)

result = vqe.run(quantum_instance)

print_result(result)

/opt/anaconda3/lib/python3.8/site-packages/qiskit/aqua/components/optimizers/optimizer.py:49: DeprecationWarning: The package qiskit.aqua.components.optimizers is deprecated. It was moved/refactored to qiskit.algorithms.optimizers (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('aqua.components.optimizers',
/opt/anaconda3/lib/python3.8/site-packages/qiskit/aqua/algorithms/vq_algorithm.py:70: DeprecationWarning: The class qiskit.aqua.algorithms.VQAlgorithm is deprecated. It was moved/refactored to qiskit.algorithms.VariationalAlgorithm (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_class('aqua.algorithms.VQAlgorithm',
/opt/anaconda3/lib/python3.8/site-packages/qiskit/aqua/quantum_instance.py:135: DeprecationWarning: The class qiskit.aqua.QuantumInstance is deprecated. It was moved/refactored to qiskit

Optimal: selection [1. 1. 0. 1. 1. 0. 1. 1. 0. 0.], value -1366.7749

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
[1 1 0 1 1 0 1 1 0 0]	-1366.7749		0.1782
[1 1 0 1 1 0 0 1 0 0]	-1365.9745		0.1212
[1 1 0 1 1 0 1 1 0 1]	-1343.9142		0.1168
[1 1 0 1 0 0 1 1 0 0]	-1291.0411		0.0821
[1 1 0 0 1 0 1 1 0 0]	-1319.1294		0.0433
[1 0 0 1 1 1 0 0 0 1]	-1270.0086		0.0416
[1 1 0 0 1 0 0 1 0 0]	-1299.5865		0.0386
[1 0 0 0 0 1 0 0 0 1]	-1073.8689		0.0351
[1 1 0 1 1 0 0 0 0 0]	-1309.6313		0.0251
[1 1 0 0 1 0 0 1 0 1]	-1317.0614		0.0211
[1 1 0 1 0 0 1 1 0 1]	-1287.4869		0.0196
[1 0 0 1 0 1 1 0 0 1]	-1164.9503		0.0194
[1 1 0 1 1 0 0 1 0 1]	-1363.4876		0.0183
[1 0 0 0 1 1 1 0 0 1]	-1172.5530		0.0169
[1 0 0 0 1 1 0 0 0 1]	-1202.4125		0.0166
[1 1 0 1 1 1 1 1 0 0]	-1450.1813		0.0129
[1 1 0 0 1 0 1 0 0 0]	-1246.3009		0.0126
[1 0 0 1 0 1 0 0 0 1]	-1161.5943		0.0121
[1 1 0 1 0 0 1 0 0 0]	-1236.8172		0.0109
[1 1 0 0 0 0 0 1 0